# Axtell Model

## Import modules

In [1]:
from __future__ import division
%pylab inline
import random as rd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


## Parameters

In [2]:
L = 5000
number_neighbors = 4
max_t = 2000
active = .1

## Agents

### Workers

In [3]:
class Worker:
    
    def __init__(self, id):
        self.id = id
        self.Theta = rd.random()
        self.employer = None
        self.effort = self.optimalEffort(0)
        self.utility = self.computeUtility(self.effort, self.effort)
        
    def optimalEffort(self, E):
        eStar = (-1 - 2*(E-self.Theta) + (1 + 4*self.Theta**2*(1+E)*(2+E))**(1/2))/(2*(1+self.Theta))
        return max([0, min([1, eStar])])
        
    def computeEffort(self, *opt_args):
        if not opt_args:
            return self.effort
        else:
            newFirm = opt_args[0]
            E = newFirm.getEfforts()
            return self.optimalEffort(E)
    
    def computeUtility(self, wage, effort):
        return (wage**self.Theta)*((1 - effort)**(1-self.Theta))
    
    def getBestNewFirm(self): # includes a startup
        # get startup firm
        startup = market.getEmptyFirm()
        # get list of neighboring firms
        neighboringFirms = market.getNeighborFirms(self)
        allNewFirms = neighboringFirms + [startup]
        allEfforts = [self.computeEffort(firm) for firm in allNewFirms]
        allSizes = [firm.getSize()+1 for firm in allNewFirms]
        allOutputs = [allNewFirms[i].getOutput(allEfforts[i]) for i in range(len(allNewFirms))]
        allUtilities = [self.computeUtility(allOutputs[i]/allSizes[i], allEfforts[i]) for i in range(len(allNewFirms))]
        bestIndex = allUtilities.index(max(allUtilities))
        return allNewFirms[bestIndex], allEfforts[bestIndex], allUtilities[bestIndex]
    
    def updateEffort(self):
        self.effort = self.optimalEffort(self.employer.getEfforts() - self.effort)
        self.utility = self.computeUtility(self.employer.getOutput()/self.employer.getSize(), self.effort)
    
    def chooseFirm(self, period):
        newFirm, newEffort, newUtility = self.getBestNewFirm()
        self.employer.updateEfforts()
        self.updateEffort()
        if newUtility > self.utility:
            market.migration(self, newFirm, period)
            self.effort = newEffort
            self.utility = newUtility
        
    def step(self, t):
        if rd.random() < active:
            self.chooseFirm(t)

### Firms

In [4]:
class Firm:
    
    def __init__(self, id):
        self.id = id
        self.book = {} # dictionary with period of hiring
    
    def hire(self, agent, period):
        self.book[agent] = period

    def separate(self, agent, period):
        self.book.pop(agent)
    
    def getEfforts(self):
        return sum([worker.effort for worker in self.book.keys()])
    
    def getOutput(self, *opt_arg):
        if not opt_arg:
            return self.getEfforts() + self.getEfforts()**2
        else:
            newEffort = opt_arg[0]
            return self.getEfforts() + newEffort + (self.getEfforts()+newEffort)**2
    
    def getSize(self):
        return len(self.book)
    
    def updateEfforts(self):
        [[worker.updateEffort() for worker in self.book.keys()] for i in range(1)]

### Market

In [5]:
class Market:
       
    def getNeighborFirms(self, agent):
        ## From network agents
        firms = list(set([neigh.employer for neigh in network[agent] if neigh.employer != agent.employer]))
        ## Pick rangom agents form the populaiton (generates Zipf law)
        if rd.random() < .01:
            firms = list(set([neigh.employer for neigh in rd.sample(workers, number_neighbors) if neigh.employer != agent.employer]))
        return firms
    
    def getEmptyFirm(self):
        return rd.choice([firm for firm in firms if firm.getSize()==0])
       
    def hiring(self, worker, firm, period):
        worker.employer = firm
        firm.hire(worker, period)

    def separation(self, worker, firm, period):    
        worker.employer = None
        firm.separate(worker, period)
        
    def migration(self, worker, newFirm, period):
        self.separation(worker, worker.employer, period)
        self.hiring(worker, newFirm, period)
    
    def step(self, period):
        [worker.step(period) for worker in workers]

## Execution

### Initialization

In [ ]:
workers = [Worker(i) for i in range(L)]
firms = [Firm(i) for i in range(2*L+1)]
market = Market()
network = {}
for worker in workers:
    network[worker] = [neigh for neigh in rd.sample(workers, number_neighbors) if neigh != worker]
# Set hirings
for i in range(L):
    market.hiring(workers[i], firms[i], 0)

### Iteration

In [ ]:
for t in range(max_t):
    market.step(t)

## Plotting

In [ ]:
x = set([firm.getSize()+1 for firm in firms])
bins = 10 ** np.linspace(np.log10(min(x)), np.log10(max(x)), 25)
plt.hist([firm.getSize()+1 for firm in firms], bins=bins, log=True, facecolor='white', histtype='step')
plt.gca().set_xscale("log")
plt.xlabel('number of employees')
plt.ylabel('frequency')
plt.title('firm size distribution')